# Text Classifiers:
#### This notebook contains various classifiers that can be used on a provided text dataset.

In [701]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

import nltk
import tensorflow as tf
import keras
import sklearn
import csv

#### Enter the name of the dataset you want to test into the file_in variable:
##### The file_out dataset is the name of the dataset that will stored the number converted version of the text data. Set the name of file_out to the name of the converted dataset if you already have a dataset that has numeric representations of the dataset already instead of words. Otherwise, file_out will be the output file name that is the converted data of the file_in dataset that gets read in for classification for the tensorflow model. Also, if you already have a converted file, make sure that re_read is set to 0 so that it does not get overwritten.

In [702]:
### Enter filenames below:
file_in = "suggestions_trialA.csv"
file_out = "trial_taskA_converted.csv"
### Enter column names for data and labels below:
data_col = "comments"
label_col = "is_prompt_exists"
### Set re_read to 0 if you have a converted text file, 1 if it needs to be created
re_read = 0

In [703]:
df = pd.read_csv(file_in, engine = 'python');
print(file_in)
print(df.dtypes)
print("Sample size:", len(df))
print(df.head(5))

suggestions_trialA.csv
id                  object
comments            object
is_prompt_exists     int64
dtype: object
Sample size: 2500
      id                                           comments  is_prompt_exists
0  663_3  Please enable removing language code from the ...                 1
1  663_4  Note: in your .csproj file, there is a Support...                 0
2  664_1  Wich means the new version not fully replaced ...                 0
3  664_2  Some of my users will still receive the old xa...                 0
4  664_3  The store randomly gives the old xap or the ne...                 0


#### Setting up the data.

In [704]:
X = df[data_col]
Y = df[label_col]

In [705]:
X_train, X_test, Y_train, Y_test = train_test_split(
 X, Y, test_size=0.4, random_state=42)

## Multinomial Naive Bayes classifier:

In [706]:
#### Multinomial Naive Bayes
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
 ('tfidf', TfidfTransformer()),
 ('clf', MultinomialNB()),
])

In [707]:
text_clf = text_clf.fit(X_train,Y_train) # X and Y switched?

In [708]:
predicted = text_clf.predict(X_test)
accuracy = np.mean(predicted == Y_test)
print("Accuracy:", accuracy)

Accuracy: 0.747


In [709]:
### Add graphics here

## Support Vector Machine classifier:

In [710]:
#### Support Vector Machine
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge',
                                                  penalty='l2',
                                                  max_iter=5,
                                                  random_state=42)), 
                    ])

In [711]:
_ = text_clf_svm.fit(X_train,Y_train)

In [712]:
predicted_svm = text_clf_svm.predict(X_test)
accuracy = np.mean(predicted_svm == Y_test)
print("Accuracy:", accuracy)

Accuracy: 0.821


In [713]:
### Add graphics here

## Gridsearch classifier:

In [714]:
#### Gridsearch
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf-svm__alpha': (1e-2, 1e-3),
    }

In [715]:
## Can use on previous svm or bayes classifier
gs_clf = GridSearchCV(text_clf_svm, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train,Y_train)
predicted_gs = gs_clf.predict(X_test)
accuracy = np.mean(predicted_gs == Y_test)

In [716]:
print("Grid search best score:", gs_clf.best_score_)
print(gs_clf.best_params_)
print("Accuracy:", accuracy)

Grid search best score: 0.836
{'clf-svm__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Accuracy: 0.814


In [717]:
### Add graphics here

## Tensorflow/Keras Classifier

#### Create a numeric version of this text data set if not present (uncomment below if to do so). Also determines length of vocabulary to use for embedding layer size:

In [718]:
# Create the word mappings
def bagofwords():
    a = []
    loop = 0
    for loop in range(num_data):
        comment = df[data_col][loop]
        sentTok = nltk.sent_tokenize(comment)
        for sentence in sentTok:
            a.append(sentence)    
    a = (' '.join(a)).lower()
    text = nltk.word_tokenize(a)
    newset = set()
    for word in text:
        newset.add(word)
    # Fill in found words
    word_to_ix = { w:(i+4) for i,w in enumerate(sorted(newset)) }
    ix_to_word = { (i+4):w for i,w in enumerate(sorted(newset)) }
    # Fill in reserved values
    ix_to_word[0] = "<PAD>"
    ix_to_word[1] = "<START>"
    ix_to_word[2] = "<UNK>"
    ix_to_word[3] = "<UNUSED>"
    word_to_ix["<PAD>"] = 0 # Used to equalize text length
    word_to_ix["<START>"] = 1
    word_to_ix["<UNK>"] = 2  # unknown value
    word_to_ix["<UNUSED>"] = 3
    return word_to_ix, ix_to_word

In [719]:
word_to_ix, ix_to_word = bagofwords()
# print(ix_to_word[3853])
# print(word_to_ix["please"]) # Ensure word is in dictionary
print("Size of vocabulary:", len(ix_to_word))

Size of vocabulary: 5752


##### An update text will be printed in order to keep track of the conversion process.

In [720]:
### Convert original data to number representations
if (re_read == 1):
    num_data = len(df)
    # num_data = 5
    loop = 0
    converted_text_data = []
    print("Total to print:", num_data)
    for loop in range(num_data):
        if (loop % 5000 == 0): # For larger datasets
            print("Update:", loop) # Check spot in converter
        comment = df[data_col][loop]
        if isinstance(comment, str) == False: # Skip missing text from dataset if present
            continue
        comment = comment.lower()
        text = nltk.word_tokenize(comment)
        length = len(text)
        i = 0
    #     print(loop, ":", text)
        for i in range(length):
    #         print(i, ':', word_to_ix[text[i]])
            try:
                text[i] = word_to_ix[text[i]]
            except KeyError:
                text[i] = 2 # Unknown mapping
    #         print(text[i]) 
    #     print(text,'\n')
    #     print('\n')
        text = [int(i) for i in text]
    #     print(text,'\n\n')
        converted_text_data.append(text)

    #     df['comments'][loop] = text

In [721]:
if (re_read == 1):
    # print(converted_text_data[0])
    with open(file_out, 'w', newline = '') as f:
        writer = csv.writer(f)
        writer.writerow([data_col, label_col])
        i = 0
        for i in range(num_data):
    #         rev = datfram['Review'][i]
            rev = converted_text_data[i]
            label = df[label_col][i]
    #         pol = (datfram['min_politeness'][i] + datfram['max_politeness'][i]) / 2.0
    #         pol = int(round(pol, 0))
    #         form = (datfram['min_formality'][i] + datfram['max_formality'][i]) / 2.0
    #         form = int(round(form, 0))
            writer.writerow([rev, label])

In [722]:
if (re_read == 1):
    print(len(df))
    print(len(converted_text_data))

#### Read in the converted text to number dataset and run the classifier:

In [723]:
df = pd.read_csv(file_out, engine = 'python');
print(file_out)
print(df.dtypes)
print("Sample size:", len(df))

trial_taskA_converted.csv
comments            object
is_prompt_exists     int64
dtype: object
Sample size: 2500


In [724]:
### Parses the integer list data from file into a proper integer list for the classifier
full_data = []
for dat in df[data_col]:
    paragraph = []
    sentence = []
    for letter in dat:
        if (letter == '[' or letter == ']' or letter == ' '):
            continue
        elif (letter == ','):
            sentence = (''.join(sentence))
            paragraph.append(sentence)
    #         print(sentence)
            sentence = []
        else:
            sentence.append(letter)
#     paragraph = (' '.join(paragraph))
    paragraph = [int(i) for i in paragraph]
    full_data.append(paragraph)
# print(paragraph, '\n')
# full_data = traindata
print("Dataset length:", len(full_data))

Dataset length: 2500


In [725]:
### Initialize train and test data, train and test labels
num_data = len(df)
num_train = int(num_data * 0.8) # df.head this amount
num_test = int(num_data - num_train) # df.tail this amount
train_data = full_data[:num_train]
test_data = full_data[num_train:]
train_labels = df[label_col].head(num_train) # is_prompt_exists
test_labels = df[label_col].tail(num_test)
print("Number train data:", len(train_data))
print("Number test data:", len(test_data))

Number train data: 2000
Number test data: 500


In [726]:
### Find the maximum sentence length to use for padding training data
maxlength = 0
for i in range(len(full_data)):
    maxlength = max(len(full_data[i]), maxlength)
print("Max text length:", maxlength)

Max text length: 515


In [727]:
### Pad the words to standardized text array length
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value=word_to_ix["<PAD>"],
                                                       padding='post',
                                                       maxlen=maxlength)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                      value=word_to_ix["<PAD>"],
                                                      padding='post',
                                                      maxlen=maxlength)

In [728]:
#### Initiate validation sets
val_size = int(num_train * 0.8) # Set apart 20% of train data for validation

x_val = train_data[:num_train-val_size]
partial_x_train = train_data[num_train-val_size:]

y_val = train_labels[:num_train-val_size]
partial_y_train = train_labels[num_train-val_size:]

print("Training data length:", len(partial_x_train))
print("Validation data length:",len(x_val))

Training data length: 1600
Validation data length: 400


In [729]:
### Create the model
import operator # Finding vocabulary length, max dictionary index + 1
vocab_size = max(ix_to_word.items(), key=operator.itemgetter(1))[0] + 1

model = keras.Sequential() 
model.add(keras.layers.Embedding(vocab_size, 16)) # original right parameter is 16
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu)) # original left parameter  is 16
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, None, 16)          92032     
_________________________________________________________________
global_average_pooling1d_16  (None, 16)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 17        
Total params: 92,321
Trainable params: 92,321
Non-trainable params: 0
_________________________________________________________________


In [730]:
### Configured model with optimizer and loss function
model.compile(optimizer = tf.train.AdamOptimizer(),
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

In [731]:
# sent_set = []
# for i in range(len(full_data)):
#     sentence = full_data[i]
#     for word in sentence:
#         sent_set.append(word)

# # wor = (' '.join(sent_set[0]))
# print(len(sent_set))
# new_set = set(sent_set)
# print(len(new_set),'\n')
# #5692
# import operator
# print(max(ix_to_word.items(), key=operator.itemgetter(1))[0])
# for word in new_set:
#     print(word)
# print(word_to_ix[5741]) ## 5741
# print(word_to_ix) ## 5751

In [732]:
### Train the model
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=40,
                   batch_size=256,
                   validation_data=(x_val, y_val),
                   verbose=1)

Train on 1600 samples, validate on 400 samples
Epoch 1/40
1600/1600 [==============================] - 1s 542us/step - loss: 0.6911 - acc: 0.5994 - val_loss: 0.6857 - val_acc: 0.8050
Epoch 2/40
1600/1600 [==============================] - 0s 44us/step - loss: 0.6838 - acc: 0.7863 - val_loss: 0.6786 - val_acc: 0.8050
Epoch 3/40
1600/1600 [==============================] - 0s 44us/step - loss: 0.6766 - acc: 0.7863 - val_loss: 0.6702 - val_acc: 0.8050
Epoch 4/40
1600/1600 [==============================] - 0s 42us/step - loss: 0.6684 - acc: 0.7863 - val_loss: 0.6611 - val_acc: 0.8050
Epoch 5/40
1600/1600 [==============================] - 0s 39us/step - loss: 0.6596 - acc: 0.7863 - val_loss: 0.6511 - val_acc: 0.8050
Epoch 6/40
1600/1600 [==============================] - 0s 37us/step - loss: 0.6500 - acc: 0.7863 - val_loss: 0.6400 - val_acc: 0.8050
Epoch 7/40
1600/1600 [==============================] - 0s 39us/step - loss: 0.6390 - acc: 0.7863 - val_loss: 0.6267 - val_acc: 0.8050
Epoch 8

In [733]:
### Evaluate the model
results = model.evaluate(test_data, test_labels)
print(results)

500/500 [==============================] - 0s 35us/step
[0.6662123136520386, 0.6700000009536743]


In [734]:
########### Graph the data
history_dict = history.history
history_dict.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])